In [1]:
from horton import *
import numpy as np
from numpy.linalg import norm
from numpy import dot,cross
Methane=IOData.from_file('Methane.xyz')
#print Methane.numbers
arrayGeom= np.asarray(Methane.coordinates)

In [2]:
def uhf (mol,basis_set='sto-3g'):
    obasis=get_gobasis(mol.coordinates,mol.numbers,basis_set)

    lf=DenseLinalgFactory(obasis.nbasis)
    #orbital integrals
    olp=obasis.compute_overlap(lf)
    kin=obasis.compute_kinetic(lf)
    na=obasis.compute_nuclear_attraction(mol.coordinates,mol.pseudo_numbers,lf)
    er=obasis.compute_electron_repulsion(lf)

    exp_alpha=lf.create_expansion()
    exp_beta=lf.create_expansion()

    guess_core_hamiltonian(olp, kin, na, exp_alpha,exp_beta)
    occ_model=AufbauOccModel(7,7)
    occ_model.assign(exp_alpha, exp_beta)

    external = {'nn': compute_nucnuc(mol.coordinates, mol.pseudo_numbers)}
    terms = [
        UTwoIndexTerm(kin, 'kin'),
        UDirectTerm(er, 'hartree'),
        UExchangeTerm(er, 'x_hf'),
        UTwoIndexTerm(na, 'ne'),
    ]
    ham = UEffHam(terms, external)
    dm_alpha = exp_alpha.to_dm()
    dm_beta = exp_beta.to_dm()
    # - SCF solver
    scf_solver = EDIIS2SCFSolver(1e-5,maxiter=300)
    scf_solver(ham, lf, olp, occ_model, dm_alpha, dm_beta)
    return ham.cache['energy']

In [3]:
class repres:
    cartesian=np.ndarray
    nucl_masses=[]
    derivatives=[]
#    Internal=np.array
    def __init__(self,molecule):
        self.molecule=molecule
        self.cartesian=np.asarray(molecule.coordinates)
        self.cartesian-=self.cartesian[0]
        self.Internal=np.zeros(self.cartesian.shape[0]*3-6)  ### for the future check linearity of the molecule 
        self.total_mass=0.
        for i in range(len(molecule.numbers)):
            self.nucl_masses.append(periodic[molecule.numbers[i]].mass)
            self.total_mass+=periodic[molecule.numbers[i]].mass
    def get_center_of_mass(self):
        mass_center=np.zeros(3)
        for i in range(len(self.nucl_masses)):
            mass_center+=self.nucl_masses[i]/self.total_mass*self.cartesian[i]
        return mass_center
    def center(self):
        cm=self.get_center_of_mass()
        for i in self.cartesian:
            i-=cm
    def update_IC(self):
        i=self.cartesian.shape[0]
        if i==2:
            self.Internal[0]=np.linalg.norm(self.cartesian[1]-self.cartesian[0])
            return
        elif i==3:
            self.Internal[0]=np.linalg.norm(self.cartesian[1])
            self.Internal[1]=np.linalg.norm(self.cartesian[2])
            self.Internal[2]=np.dot(self.cartesian[2],self.cartesian[1])
            return
        elif i>=4:
            self.Internal[0]=np.linalg.norm(self.cartesian[1])
            self.Internal[1]=np.linalg.norm(self.cartesian[2])
            self.Internal[2]=np.dot(self.cartesian[2],self.cartesian[1])/self.Internal[0]/self.Internal[1]
            for j in range(3,i):
                self.Internal[3*(j-2)]=norm(self.cartesian[j])
                self.Internal[3*(j-2)+1]=np.dot(self.cartesian[j],self.cartesian[j-1])\
                /norm(self.cartesian[j])/norm(self.cartesian[j-1])
                self.Internal[3*(j-2)+2]=dot(self.cartesian[j],cross(self.cartesian[j-1],self.cartesian[j-2]))\
                /norm(self.cartesian[j])/norm(cross(self.cartesian[j-1],self.cartesian[j-2]))   
    def update_Cart(self):
        self.cartesian[0]=[0,0,0]
        self.cartesian[1]=[self.Internal[0],0,0]
        self.cartesian[2]=[self.Internal[1]*self.Internal[2],self.Internal[1]*(1-self.Internal[2]**2)**0.5,0]
        
        for i in range (3,self.cartesian.shape[0]):
            a=self.cartesian[i-1]/norm(self.cartesian[i-1])*self.Internal[3*(i-2)+1]
            b=cross(self.cartesian[i-1],self.cartesian[i-2])/norm(cross(self.cartesian[i-1],self.cartesian[i-2]))*self.Internal[3*(i-2)+2]
            c=cross(cross(self.cartesian[i-1],self.cartesian[i-2]),self.cartesian[i-1])*(1-self.Internal[3*(i-2)+2]**2-self.Internal[3*(i-2)+1]**2)**0.5\
                                          /norm(cross(cross(self.cartesian[i-1],self.cartesian[i-2]),self.cartesian[i-1]))
            self.cartesian[i]=self.Internal[3*(i-2)]*(a+b+c)
        self.molecule.coordinates=self.cartesian.tolist()
    
    def derive(self,cn,h):
        e0=uhf(self.molecule)
        dHV=np.zeros(self.Internal.shape)
        dHV[cn]=h
        self.Internal+=dHV #e(x+h)
        self.update_Cart()
        e1=uhf(self.molecule)
        self.Internal-=2*dHV #e(x-h)
        self.update_Cart()
        e2=uhf(self.molecule)
        f1=(e1-e2)/2./h
        f2=(e1-2*e0+e2)/h**2
        # --- move to next point ---
        dx=-f1/f2
        self.Internal+=dHV*(1+dx/h)
        self.update_Cart()
        print f1,f2,'  dx ',dx,'*******************************************************'
        return (f1,f2)
    
    def optimiz(self):
 #       print uhf(self.molecule),'_______________HJHJSHJA______'
        opt_coords=[]
        for x in range (self.Internal.shape[0]):
            if x in opt_coords: pass
            else:
                i=self.derive (x,0.0001)
                if abs(i[0])<1.e-8:
                    opt_coords.append(x)
        return uhf(self.molecule),'_______________HJHJSHJA______'
#        return uhf(self.molecule)
                
        
    def get_cartesian(self):
        return self.cartesian.tolist()
    
    


In [ ]:
r=repres(Methane)


In [ ]:

print "************************"
print r.get_cartesian()
print r.Internal 
ff=r.derive(0,.001)
#r.optimiz()

************************
[[0.0, 0.0, 0.0], [2.0639588834687914, 0.0, 0.0], [-0.6879736963153711, -1.6521497643646723, -1.0281244004211965], [-0.6879736963153711, 1.7164382474406734, -0.9167439420878778], [-0.6879736963153711, -0.06430738033734018, 1.944849445247735]]
[0. 0. 0. 0. 0. 0. 0. 0. 0.]
 _ __ _
/ (..) \ Welcome to HORTON 2.1.1!
\/ || \/
 |_''_|  HORTON is written and maintained by by Toon Verstraelen (1).

         This version contains contributions from Toon Verstraelen (1), Pawel Tecmer (2),
         Farnaz Heidar-Zadeh (2), Cristina E. González-Espinoza (2), Matthew Chan (2),
         Taewon D. Kim (2), Katharina Boguslawski (2), Stijn Fias (3),
         Steven Vandenbrande (1), Diego Berrocal (2), and Paul W. Ayers (2)

         (1) Center for Molecular Modeling (CMM), Ghent University, Ghent, Belgium.
         (2) The Ayers Group, McMaster University, Hamilton, Ontario, Canada.
         (3) General Chemistry (ALGC), Free University of Brussels, Brussels, Belgium.

      

/home/giorgiod/.local/lib/python2.7/site-packages/horton/quadprog.py:621: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  lagrange = np.linalg.lstsq(r_free.T, -g_free)[0]
